In [1]:
import openai
import whisper
from transformers import pipeline
import yt_dlp as youtube_dl
from bertopic import BERTopic
from tqdm import tqdm

In [2]:
# Function to download the audio using yt-dlp
def download_audio(video_link, output_audio_path):
    ydl_opts = {
        "format": "bestaudio/best",
        "postprocessors": [{"key": "FFmpegExtractAudio", "preferredcodec": "wav", "preferredquality": "192"}],
        "postprocessor_args": ["-ar", "16000"],
        "prefer_ffmpeg": True,
        "keepvideo": False,
        "outtmpl": output_audio_path + ".%(ext)s",
    }
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
        ydl.download([video_link])

In [3]:
# Function to transcribe audio and extract segments using Whisper
def transcribe_audio_with_timestamps(audio_path, model_name="base"):
    model = whisper.load_model(model_name)
    print("Transcribing audio...")
    result = model.transcribe(audio_path)
    transcription = result["text"]
    segments = result["segments"]  # Contains timestamps per segment
    return transcription, segments

In [4]:
# Summarization using transformer-based summarization (e.g., BART)
def summarize_text_in_chunks(text, chunk_size=1024):
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=-1)

    # Split text into chunks
    text_chunks = [text[i : i + chunk_size] for i in range(0, len(text), chunk_size)]
    summaries = []

    for chunk in text_chunks:
        summary = summarizer(chunk, max_length=150, min_length=40, do_sample=False)[0]["summary_text"]
        summaries.append(summary)

    return " ".join(summaries)

In [5]:
# Set up OpenAI API Key for ChatGPT
openai.api_key = "your_openai_api_key_here"


# Function to ask questions using RAG-like structure with ChatGPT API
def query_chatgpt(context, question):
    response = openai.Completion.create(engine="gpt-4", prompt=f"{context}\n\nQ: {question}\nA:", max_tokens=150, temperature=0.5)
    return response.choices[0].text.strip()

In [6]:
# Function to dynamically segment the transcription into topics using BERTopic
def topic_segmentation(segments):
    texts = [segment["text"] for segment in segments]
    topic_model = BERTopic()  # Initialize BERTopic model

    print("Segmenting topics...")
    # Use tqdm to show progress while segmenting the topics
    topics, _ = topic_model.fit_transform(tqdm(texts, desc="Topic segmentation"))

    # Group segments by topics with timeframes
    topic_segments = {}
    for idx, topic in enumerate(topics):
        if topic not in topic_segments:
            topic_segments[topic] = {"text": [], "start_time": segments[idx]["start"], "end_time": segments[idx]["end"]}
        topic_segments[topic]["text"].append(segments[idx]["text"])
        topic_segments[topic]["end_time"] = segments[idx]["end"]  # Update the end time

    return topic_segments, topic_model

In [7]:
# Generate summaries for each segmented topic
def summarize_topics(topic_segments, chunk_size=1024):
    topic_summaries = {}
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=-1)  # Use CPU by setting device=-1

    print("Summarizing topics...")
    for topic, data in tqdm(topic_segments.items(), desc="Summarizing each topic"):
        full_text = " ".join(data["text"])  # Combine all texts for this topic

        # Split the full text into chunks to handle large inputs
        text_chunks = [full_text[i : i + chunk_size] for i in range(0, len(full_text), chunk_size)]
        summaries = []

        for chunk in text_chunks:
            summary = summarizer(chunk, max_length=150, min_length=40, do_sample=False)[0]["summary_text"]
            summaries.append(summary)

        # Combine chunk summaries and store them
        topic_summaries[topic] = {"summary": " ".join(summaries), "start_time": data["start_time"], "end_time": data["end_time"]}

    return topic_summaries

In [8]:
# Main function to process video, transcribe, segment by topic, summarize, and enable Q&A with ChatGPT
def process_lecture_video(video_link, audio_output_path):
    # Step 1: Download the audio from the video
    print("STEP 1:....")
    download_audio(video_link, audio_output_path)

    # Step 2: Transcribe the audio and get timestamps for each segment
    print("\nSTEP 2:....")
    transcription, segments = transcribe_audio_with_timestamps(audio_output_path + ".wav")

    # Step 3: Generate overall summary of the lecture
    print("\nSTEP 3:....")
    overall_summary = summarize_text_in_chunks(transcription, chunk_size=1024)

    # Step 4: Segment the transcription by topics dynamically
    print("\nSTEP 4:....")
    topic_segments, topic_model = topic_segmentation(transcription, segments)

    # Step 5: Summarize each segmented topic
    print("\nSTEP 5:....")
    topic_summaries = summarize_topics(topic_segments)

    return overall_summary, topic_summaries, topic_model

In [9]:
# Function to handle queries, providing both answer and relevant timeframe
def query_with_timeframe(question, transcript, segments, topic_summaries):
    # Answer the question using ChatGPT API
    answer = query_chatgpt(transcript, question)

    # Find relevant topic timeframe from answer
    relevant_timeframe = None
    for topic, summary in topic_summaries.items():
        if answer.lower() in summary["summary"].lower():
            relevant_timeframe = (summary["start_time"], summary["end_time"])
            break

    return answer, relevant_timeframe

In [10]:
# Example Usage:
video_link = "https://www.youtube.com/watch?v=AhyznRSDjw8"
audio_output_path = "downloaded_audio"

In [11]:
# # Process the lecture video to get summaries and topics
# overall_summary, topic_summaries, topic_model = process_lecture_video(video_link, audio_output_path)

In [12]:
# Step 1: Download the audio from the video
print("STEP 1:....")
download_audio(video_link, audio_output_path)

STEP 1:....
[youtube] Extracting URL: https://www.youtube.com/watch?v=AhyznRSDjw8
[youtube] AhyznRSDjw8: Downloading webpage
[youtube] AhyznRSDjw8: Downloading ios player API JSON
[youtube] AhyznRSDjw8: Downloading web creator player API JSON
[youtube] AhyznRSDjw8: Downloading m3u8 information
[info] AhyznRSDjw8: Downloading 1 format(s): 251
[download] Destination: downloaded_audio.webm
[download] 100% of   45.54MiB in 00:00:03 at 12.82MiB/s    
[ExtractAudio] Destination: downloaded_audio.wav
Deleting original file downloaded_audio.webm (pass -k to keep)


In [13]:
# Step 2: Transcribe the audio and get timestamps for each segment
print("STEP 2:....")
transcription, segments = transcribe_audio_with_timestamps(audio_output_path + ".wav")

STEP 2:....
Transcribing audio...


In [14]:
# Step 3: Generate overall summary of the lecture
print("STEP 3:....")
overall_summary = summarize_text_in_chunks(transcription, chunk_size=1024)

STEP 3:....


In [16]:
# Step 4: Segment the transcription by topics dynamically
print("STEP 4:....")
topic_segments, topic_model = topic_segmentation(segments)

STEP 4:....
Segmenting topics...


Topic segmentation: 100%|██████████| 750/750 [00:00<00:00, 748270.22it/s]


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:
# Step 5: Summarize each segmented topic
print("STEP 5:....")
topic_summaries = summarize_topics(topic_segments)

STEP 5:....
Summarizing topics...


Summarizing each topic: 100%|██████████| 20/20 [05:00<00:00, 15.04s/it]


In [18]:
# Output the overall summary and topic summaries
print("Overall Summary of the lecture:", overall_summary)
for topic, summary in topic_summaries.items():
    print(f"Topic {topic}: {summary['summary']} (Time: {summary['start_time']} to {summary['end_time']})")

Overall Summary of the lecture: Two lectures today are really exciting because they start to move beyond a lot of what we've talked about in the class so far, which is focusing a lot on really static data sets. In today, in this lecture right now, I'm going to start to talk about how we can learn about this very long-standing field of reinforcement learning. In the real world, you have your deep learning model actually deployed together with the data. This is the key motivation of reinforcement learning. You're going to try and learn through reinforcement, making mistakes in your world, and then collecting data on those mistakes to learn how to improve. "I'm hoping for a 5-0 not to lose any games, but I think the realistic goal would be 4-1 in my favour" "I wasn't expecting that good. Everything that he did was proper. It was calculated and it was done well" We've really covered two different types of learning in this course to date. Supervised learning is in this domain where we're gi

In [19]:
# Example: Ask a query and get answer with timeframe
question = "What is the main topic discussed?"
answer, timeframe = query_with_timeframe(question, overall_summary, topic_summaries)
print(f"Answer: {answer}, Relevant Timeframe: {timeframe}")

TypeError: query_with_timeframe() missing 1 required positional argument: 'topic_summaries'